# 1. 공사정보 목록에서 "SPTNO" 추출

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질계획 현황"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질계획 현황/selectQltPlnList.csv'

# 2. 연도별 공사계약 목록에서 "SPTTO" 추출 * SPTNO 활용

In [2]:
targetData

,순서,자료대상,종류,자료명,데이터 종류,작업자명,망 확인필요,수집방법,컬럼명,기본키,요청변수,저장폴더,전처리 항목,업데이트주기,서비스키명,URL,서비스키,파라미터,제공방식,데이터타입,참고문서,참고문서타입,주기,비고
2,3,건설사업정보시스템 - 공사,INBOUND,품질계획 현황,WEB,박동주,외부망,openAPI 연계,"공사정보 : sptNo, odofBrn, cno, cwkNm, orcd, ornm,...","serviceKey,sptNo,rprtYm","serviceKey,sptNo,rprtYm,type",../output/건설사업정보시스템/품질계획 현황/selectQltPlnList.csv,-,-,-,https://www.calspia.go.kr/io/openapi/cm/select...,selectQltPlnList.do,serviceKey=&sptNo=&rprtYm=&type=json,api,json,건설사업정보시스템-공사,Excel,-,NaN


In [3]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/연도별 공사계약 목록/selectIoCmProjConstYearContractList.csv", encoding="ms949")

In [4]:
dfCombi = imsiDf.loc[:,["sptno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombi.head(10)


,sptno
0,C2010007
1,C2013925
2,C2004186
3,C2019022
4,C2003095
5,C2003142
6,C1997009
7,C2004031
8,C2003112
9,C2005069


In [5]:
dfCombiLen = dfCombi.shape[0]
dfCombiLen

1222

In [6]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectQltPlnList'

In [7]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
PRIMARYKEY

'serviceKey,sptNo,rprtYm'

# 3. 기성보고 현황 수집

In [8]:
JSONKEY = "detail1"
DUMMY = 0

In [9]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN

0

In [10]:
from datetime import datetime

# d1, d2 : datetime format 
# d1 >= d2
def calc_month(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month + 1)

In [ ]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()
for i in range(0,dfCombiLen):
    sptnoValue = dfCombi.loc[i].sptno
    print(sptnoValue)
    BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
    resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
    else:  
        tmpStartDay = resultDf["stwrdt"].values[0].split('-')
        tmpEndDay = resultDf["ccwdt"].values[0].split('-')

        dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y-%m').tolist()
        print(dateRange)
        for i in range(0,len(dateRange)):
            BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
            resultDf = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
            resultDfMerged = resultDfMerged.append(resultDf) 
            print(resultDf)
        

C2010007
1 page scraping start
Empty DataFrame
Columns: []
Index: []
1 page is empty
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2010007', 'rprtYm': '', 'type': 'json'} rows: 0 completed
No Data
C2013925
1 page scraping start
Empty DataFrame
Columns: []
Index: []
1 page is empty
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2013925', 'rprtYm': '', 'type': 'json'} rows: 0 completed
No Data
C2004186
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '', 'type': 'json'} rows: 24 completed
['2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2009-01', '2009-02', '2009-03', '2009-04', '2009-05', '2009-06', '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12', '2010-01', '

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2008-10', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C2004186  C2004193  가금제 개수공사    S  서울청    A1    한강  경기도 김포시   

  bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn ccwynnm bzarcd  \
0     A1     한강     01  2007-12-27  2012-12-26  20121226     1    완료공사    102   

  bzarnm extcyclsecd extcyclsenm         lastamt        cwksctnnm  index  
0     하천           C          시공  12,211,532,000  하성면 시암리~월곶면 용강리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2008-11', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C20

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2009-10', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C2004186  C2004193  가금제 개수공사    S  서울청    A1    한강  경기도 김포시   

  bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn ccwynnm bzarcd  \
0     A1     한강     01  2007-12-27  2012-12-26  20121226     1    완료공사    102   

  bzarnm extcyclsecd extcyclsenm         lastamt        cwksctnnm  index  
0     하천           C          시공  12,211,532,000  하성면 시암리~월곶면 용강리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2009-11', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C20

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2010-10', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C2004186  C2004193  가금제 개수공사    S  서울청    A1    한강  경기도 김포시   

  bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn ccwynnm bzarcd  \
0     A1     한강     01  2007-12-27  2012-12-26  20121226     1    완료공사    102   

  bzarnm extcyclsecd extcyclsenm         lastamt        cwksctnnm  index  
0     하천           C          시공  12,211,532,000  하성면 시암리~월곶면 용강리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2010-11', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C20

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2011-10', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C2004186  C2004193  가금제 개수공사    S  서울청    A1    한강  경기도 김포시   

  bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn ccwynnm bzarcd  \
0     A1     한강     01  2007-12-27  2012-12-26  20121226     1    완료공사    102   

  bzarnm extcyclsecd extcyclsenm         lastamt        cwksctnnm  index  
0     하천           C          시공  12,211,532,000  하성면 시암리~월곶면 용강리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2004186', 'rprtYm': '2011-11', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno     cwknm orcd ornm rutcd rutnm   pdznnm  \
0  1108300348  C20

...

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2007-09', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  2248301909  C2003112  C2003112  간성-현내 도로건설공사    W  원주청   007   

                  rutnm   pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 7호선(부산 ~ 온성 (간성))  강원도 고성군      B   지역간선     00  2003-06-12  2016-12-31   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20161231     1    완료공사    101     도로           C          시공   

          lastamt                   cwksctnnm  index  
0  95,764,588,000  강원도 고성군 간성읍 상리~고성군 현내면 대진리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2007-10', 'type': 'json'} rows: 24 completed
      odofbrn     sptno

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2008-08', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  2248301909  C2003112  C2003112  간성-현내 도로건설공사    W  원주청   007   

                  rutnm   pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 7호선(부산 ~ 온성 (간성))  강원도 고성군      B   지역간선     00  2003-06-12  2016-12-31   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20161231     1    완료공사    101     도로           C          시공   

          lastamt                   cwksctnnm  index  
0  95,764,588,000  강원도 고성군 간성읍 상리~고성군 현내면 대진리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2008-09', 'type': 'json'} rows: 24 completed
      odofbrn     sptno

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2009-07', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  2248301909  C2003112  C2003112  간성-현내 도로건설공사    W  원주청   007   

                  rutnm   pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 7호선(부산 ~ 온성 (간성))  강원도 고성군      B   지역간선     00  2003-06-12  2016-12-31   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20161231     1    완료공사    101     도로           C          시공   

          lastamt                   cwksctnnm  index  
0  95,764,588,000  강원도 고성군 간성읍 상리~고성군 현내면 대진리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2009-08', 'type': 'json'} rows: 24 completed
      odofbrn     sptno

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2010-06', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  2248301909  C2003112  C2003112  간성-현내 도로건설공사    W  원주청   007   

                  rutnm   pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 7호선(부산 ~ 온성 (간성))  강원도 고성군      B   지역간선     00  2003-06-12  2016-12-31   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20161231     1    완료공사    101     도로           C          시공   

          lastamt                   cwksctnnm  index  
0  95,764,588,000  강원도 고성군 간성읍 상리~고성군 현내면 대진리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2010-07', 'type': 'json'} rows: 24 completed
      odofbrn     sptno

Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2011-05', 'type': 'json'} rows: 24 completed
      odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  2248301909  C2003112  C2003112  간성-현내 도로건설공사    W  원주청   007   

                  rutnm   pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 7호선(부산 ~ 온성 (간성))  강원도 고성군      B   지역간선     00  2003-06-12  2016-12-31   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20161231     1    완료공사    101     도로           C          시공   

          lastamt                   cwksctnnm  index  
0  95,764,588,000  강원도 고성군 간성읍 상리~고성군 현내면 대진리      0  
1 page scraping start
Empty DataFrame
Columns: []
Index: []
0 no pageNo
dataframe품질계획 현황, param:{'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'sptNo': 'C2003112', 'rprtYm': '2011-06', 'type': 'json'} rows: 24 completed
      odofbrn     sptno

In [12]:
resultDfMerged

,odofbrn,sptno,cno,cwknm,orcd,ornm,rutcd,rutnm,pdznnm,bzkdcd,bzkdnm,bztocd,stwrdt,ccwdt,ccwxpcdt,ccwyn,ccwynnm,bzarcd,bzarnm,extcyclsecd,extcyclsenm,lastamt,cwksctnnm,index
0,1108300348,C2004186,C2004193,가금제 개수공사,S,서울청,A1,한강,경기도 김포시,A1,한강,01,2007-12-27,2012-12-26,20121226,1,완료공사,102,하천,C,시공,"12,211,532,000",하성면 시암리~월곶면 용강리,0
0,1108300348,C2004186,C2004193,가금제 개수공사,S,서울청,A1,한강,경기도 김포시,A1,한강,01,2007-12-27,2012-12-26,20121226,1,완료공사,102,하천,C,시공,"12,211,532,000",하성면 시암리~월곶면 용강리,0
0,1108300348,C2004186,C2004193,가금제 개수공사,S,서울청,A1,한강,경기도 김포시,A1,한강,01,2007-12-27,2012-12-26,20121226,1,완료공사,102,하천,C,시공,"12,211,532,000",하성면 시암리~월곶면 용강리,0
0,1108300348,C2004186,C2004193,가금제 개수공사,S,서울청,A1,한강,경기도 김포시,A1,한강,01,2007-12-27,2012-12-26,20121226,1,완료공사,102,하천,C,시공,"12,211,532,000",하성면 시암리~월곶면 용강리,0
0,1108300348,C2004186,C2004193,가금제 개수공사,S,서울청,A1,한강,경기도 김포시,A1,한강,01,2007-12-27,2012-12-26,20121226,1,완료공사,102,하천,C,시공,"12,211,532,000",하성면 시암리~월곶면 용강리,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3068301763,C2005069,C2005073,갈산-해미(서천판교우회) 도로건설공사,D,대전청,026,국도 26호선(군산 ~ 대구),충청남도 서천군,A,일반국도,01,2005-12-20,2014-07-23,20140723,1,완료공사,101,도로,C,시공,"79,607,000,000",충남 서천군 문곡리~충남 서천군 복대리,0
0,3068301763,C2005069,C2005073,갈산-해미(서천판교우회) 도로건설공사,D,대전청,026,국도 26호선(군산 ~ 대구),충청남도 서천군,A,일반국도,01,2005-12-20,2014-07-23,20140723,1,완료공사,101,도로,C,시공,"79,607,000,000",충남 서천군 문곡리~충남 서천군 복대리,0
0,3068301763,C2005069,C2005073,갈산-해미(서천판교우회) 도로건설공사,D,대전청,026,국도 26호선(군산 ~ 대구),충청남도 서천군,A,일반국도,01,2005-12-20,2014-07-23,20140723,1,완료공사,101,도로,C,시공,"79,607,000,000",충남 서천군 문곡리~충남 서천군 복대리,0
0,3068301763,C2005069,C2005073,갈산-해미(서천판교우회) 도로건설공사,D,대전청,026,국도 26호선(군산 ~ 대구),충청남도 서천군,A,일반국도,01,2005-12-20,2014-07-23,20140723,1,완료공사,101,도로,C,시공,"79,607,000,000",충남 서천군 문곡리~충남 서천군 복대리,0


In [13]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

품질계획 현황 save compled


In [14]:
resultDfMerged.shape

(630, 24)